# Lecture 15

Reminder from Lecture 13.

In [1]:
# Create some virtual classes

class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name
    

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [2]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Algorithm")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print self.name()+" Error: Bad Grade."
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists."
            raise Exception

    def id_number(self):
        return __id_number
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print self.__grades[grade]
    


In [3]:
class grade_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade):
        raise NotImplementedError
                

class uncurved_letter_grade_percent(grade_calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        grade_calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade),
                                 False)
        

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print isinstance(a_grade,grade)
            print self.name()+ " Error: Did not get an proper grade as input."
            raise Exception
        if not a_grade.numerical():
            print self.name()+ " Error: Did not get a numerical grade as input."
            raise Exception

        percent=a_grade.value()/self.__max_grade
        
        for i,v in enumerate(self.__grades_definition):
            #print percent, i, v
            if percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
            

## Quiz 2 Solution

In [4]:
class curved_letter_grade(grade_calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        grade_calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print isinstance(a_grade,grade)
            print self.name()+ " Error: Did not get an proper grade as input."
            raise Exception
        if not a_grade.numerical():
            print self.name()+ " Error: Did not get a numerical grade as input."
            raise Exception
    
        # Rescale the grade
        percent=a_grade.value()/self.__max_grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
        scaled_percent=scale_std+0.8
        
        for i,v in enumerate(self.__grades_definition):
            #print percent, i, v
            if scaled_percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
            

In [5]:
a_student=student("John","Doe",111)

a_student.add_grade(grade("Exam 1",value=70))
a_student.add_grade(grade("Exam 1 Letter",value="A"))

a_student.print_grades()

Exam 1 Letter: A
Exam 1: 70


In [6]:
a_grader=curved_letter_grade(grade_name="Exam 1 Test Grade",mean=50.,std=10.)
print a_grader.apply(a_student["Exam 1"])

Exam 1 Test Grade: A+


In [7]:
print a_grader.apply(grade("Test Grade",value=75.))

Exam 1 Test Grade: A+


## Stats Computation

In [8]:
import numpy as np
import math

class statistics_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,grades):
        raise NotImplementedError
        
class mean_std_calculator(statistics_calculator):
    def __init__(self):
        statistics_calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,grades):
        return np.mean(grades),math.sqrt(np.var(grades))


## Grade Summing

In [9]:
class summary_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_student):
        raise NotImplementedError
        
class grade_summer(summary_calculator):
    def __init__(self,prefix,n):
        self.__prefix=prefix
        self.__n=n
        statistics_calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_student):
        labels=[prefix+str(x) for x in range(1,n)]
        
        grade_sum=0.
        for label in labels:
            grade_sum+=a_student[label]
            
        a_student.add_grade(grade(prefix+"sum",value=grade_sum))
              
              

        

## Gradebook

In [10]:
class grade_book(data):
    
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
    
    def add_student(self,a_student):
        self.__students[a_student.id_number]=a_student
        
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print self.name()+" Error: Did not find student."
            
    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.iteritems():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))
            
    def apply_stats(self,a_stat_comp,grade_name):
        grades=list()
        for k,a_student in self.__students.iteritems():
            grades.append(a_student[grade_name].value())
            
        return a_stat_comp.apply(grades)
        
    

## Creating a gradebook


In [11]:
import pandas as pd
Data=pd.read_csv("Scores.csv")

In [12]:
Data

,l1_n,l1_1,12_n,l2_1,l2_2,l2_3,l2_4,l2_5,l2_6,l2_7,...,e1_6,e1_7,e1_8,e1_9,e1_10,e1_11,e1_12,e1_13,e1_14,e1_15
0,1,10,7,10,10,9.5,9.0,10,10,9.5,...,0,0,10,10,10,10,0,10,5,10
1,1,10,7,0,10,9.5,0.0,10,10,0.0,...,0,9,9,9,9,9,10,5,0,0
2,1,10,7,10,10,0.0,10.0,10,10,10.0,...,9,7,9,10,10,10,5,10,5,0
3,1,10,7,10,10,10.0,9.5,10,10,9.5,...,0,7,5,9,9,9,0,0,0,0
4,1,10,7,10,10,5.0,9.5,10,10,9.5,...,10,0,9,10,9,10,9,5,0,0
5,1,10,7,10,10,3.0,7.0,10,10,9.0,...,10,10,10,10,10,10,10,9,8,2
6,1,10,7,10,10,3.0,9.5,10,10,9.5,...,9,0,0,10,10,9,5,10,8,0
7,1,10,7,10,10,0.0,5.0,10,10,9.5,...,0,0,0,0,0,0,0,0,0,0
8,1,10,7,0,0,0.0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,1,10,7,10,10,10.0,9.5,10,10,9.5,...,10,7,0,9,9,9,0,5,0,0


In [13]:
Data.keys()

Index([u'l1_n', u'l1_1', u'12_n', u'l2_1', u'l2_2', u'l2_3', u'l2_4', u'l2_5',
       u'l2_6', u'l2_7', u'l3_n', u'l3_1', u'l3_2', u'l3_3', u'l3_4', u'l3_5',
       u'l3_6', u'l3_7', u'l3_8', u'l3_9', u'l3_10', u'l3_11', u'l3_12',
       u'l3_13', u'l3_14', u'l4_n', u'l4_1', u'l4_2', u'l4_3', u'l4_4',
       u'l4_5', u'l4_6', u'l4_7', u'l4_8', u'l4_9', u'l4_10', u'l4_11',
       u'q1_n', u'q1_1', u'e1_n', u'e1_1', u'e1_2', u'e1_3', u'e1_4', u'e1_5',
       u'e1_6', u'e1_7', u'e1_8', u'e1_9', u'e1_10', u'e1_11', u'e1_12',
       u'e1_13', u'e1_14', u'e1_15'],
      dtype='object')

In [14]:
Data.shape

(16, 55)

In [15]:
a_grade_book=grade_book("Data 1401")

for student_i in range(Data.shape[0]):
    a_student_0=student("Student",str(student_i),student_i)

    for k in Data.keys():
        a_student_0.add_grade(grade(k,value=Data[k][student_i]))

    a_grade_book.add_student(a_student_0)
        

In [16]:
a_grade_book.apply_stats(mean_std_calculator(),"l2_3")

(5.53125, 4.364160679615268)